In [ ]:
# Training celltypist model with the Human Lung Atlas for Idiopathic Fibrosis

In [1]:
import scanpy as sc

In [1]:
# -------------------------------------------------------- Filtrado por estudios 
# Seleccionamos el sub_anndata_muchari_budinger donde study == "Mishari_Budinger_2018"


adata_subset = sc.read_h5ad("/home/mdiaz/scRNA-seq_LungProject/atlas_reference/data/processed/hlca_subset.h5ad")

sub_anndata_muchari_budinger = adata_subset[adata_subset.obs["study"] == "Misharin_Budinger_2018"].copy()
sub_anndata_complemento = adata_subset[adata_subset.obs["study"] != "Misharin_Budinger_2018"].copy()




In [2]:
sub_anndata_muchari_budinger.write_h5ad('/home/mdiaz/scRNA-seq_LungProject/atlas_reference/data/processed/SinAnotar_muchari_budinger.h5ad')

In [3]:
sub_anndata_complemento.write_h5ad('/home/mdiaz/scRNA-seq_LungProject/atlas_reference/data/processed/HCA_IPF.h5ad')

In [10]:
import pandas as pd
import numpy as np

adata = sub_anndata_complemento
df = adata.obs[["transf_ann_level_2_label", "donor_id", "lung_condition"]].copy()

# Tamaño total deseado
n_total = 500_000

# Agrupamos por la combinación de interés
group_sizes = df.groupby(["transf_ann_level_2_label", "donor_id", "lung_condition"]).size()
group_fracs = group_sizes / group_sizes.sum()
n_per_group = (group_fracs * n_total).round().astype(int)

# Submuestreo respetando combinaciones
sampled_indices = []
rng = np.random.default_rng(seed=42)

for (ctype, donor, condition), n in n_per_group.items():
    group_idx = df[
        (df["transf_ann_level_2_label"] == ctype) &
        (df["donor_id"] == donor) &
        (df["lung_condition"] == condition)
    ].index

    if len(group_idx) >= n:
        sampled = rng.choice(group_idx, size=n, replace=False)
    else:
        sampled = group_idx  # Si hay menos de las que se querían, toma todas
    sampled_indices.extend(sampled)

# Crear nuevo AnnData con la submuestra
adata_sampled = adata[sampled_indices].copy()


/tmp/ipykernel_1808126/1869327746.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_sizes = df.groupby(["transf_ann_level_2_label", "donor_id", "lung_condition"]).size()


In [11]:
adata_sampled.write_h5ad('/home/mdiaz/scRNA-seq_LungProject/atlas_reference/data/processed/HCA_IPF_medioMillon.h5ad')

In [12]:
adata_sampled 

AnnData object with n_obs × n_vars = 499898 × 56239
    obs: 'suspension_type', 'donor_id', 'is_primary_data', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'tissue_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'cause_of_death', 'core_or_extension', 'dataset', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'smoking_status', 'study', 'subject_type', 'tissue_coarse_unharmonized', '

In [ ]:
models.models_description()

In [3]:
adata = sc.read_h5ad('/home/mdiaz/scRNA-seq_LungProject/atlas_reference/data/processed/adata_millonYmedio_anotado_muchari_budinger.h5ad')

In [4]:
adata.obs['predicted_labels'].value_counts().sort_index()

predicted_labels
B cell                                                    533
CD1c-positive myeloid dendritic cell                       26
CD4-positive, alpha-beta T cell                           842
CD8-positive, alpha-beta T cell                           505
alveolar adventitial fibroblast                             1
alveolar macrophage                                      1231
alveolar type 1 fibroblast cell                            77
capillary endothelial cell                                153
classical monocyte                                       1190
club cell                                                   1
endothelial cell of lymphatic vessel                       99
mast cell                                                 496
multiciliated columnar cell of tracheobronchial tree     1356
natural killer cell                                        68
non-classical monocyte                                     44
plasma cell                                          